In [44]:
import tensorflow as tf
import functools
from tqdm import tqdm
import os
from pathlib import Path
import math
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
EPOCHS = 1000
batch_size = 32
image_size = 224
STEPS_PER_TPU_CALL = 1
learning_rate = 5e-5  # should be smaller than training on single GPU
feature_size = 2048  # Embedding size before the output layer
save_interval = 2000

# ArcFace params
margin = 0.1  # DELG used 0.1, original ArcFace paper used 0.5. When margin is 0, it should be the same as doing a normal softmax but with embedding and weight normalised.
logit_scale = int(math.sqrt(feature_size))

# GeM params
gem_p = 3.
train_p = False  # whether to learn gem_p or not

data_dir = "/home/ubuntu/Dacon/jin/NIA"

In [3]:
def NormalizeImages(images, pixel_value_scale=0.5, pixel_value_offset=0.5):
  """Normalize pixel values in image.
  Output is computed as
  normalized_images = (images - pixel_value_offset) / pixel_value_scale.
  Args:
    images: `Tensor`, images to normalize.
    pixel_value_scale: float, scale.
    pixel_value_offset: float, offset.
  Returns:
    normalized_images: `Tensor`, normalized images.
  """
  images = tf.cast(images, tf.float32)
  normalized_images = tf.math.divide(
      tf.subtract(images, pixel_value_offset), pixel_value_scale)
  return normalized_images

In [4]:
def _ParseFunction(example, name_to_features, image_size, augmentation, unique_landmark_ids):
  """Parse a single TFExample to get the image and label and process the image.
  Args:
    example: a `TFExample`.
    name_to_features: a `dict`. The mapping from feature names to its type.
    image_size: an `int`. The image size for the decoded image, on each side.
    augmentation: a `boolean`. True if the image will be augmented.
  Returns:
    image: a `Tensor`. The processed image.
    label: a `Tensor`. The ground-truth label.
  """
  parsed_example = tf.io.parse_single_example(example, name_to_features)
  # Parse to get image.
  image = parsed_example['image/encoded']
  image = tf.io.decode_jpeg(image)
  image = NormalizeImages(
      image, pixel_value_scale=128.0, pixel_value_offset=128.0)
  if augmentation:
    image = _ImageNetCrop(image)
  else:
    image = tf.image.resize(image, [image_size, image_size])
    image.set_shape([image_size, image_size, 3])
  # Parse to get label.
  label = parsed_example['image/class/label']
  #label = tf.reduce_min(tf.where(tf.equal(unique_landmark_ids, label)))

  return image, label

In [5]:
def CreateDataset(file_pattern,
                  image_size=224,
                  batch_size=32,
                  augmentation=False,
                  seed=0):
  """Creates a dataset.
  Args:
    file_pattern: str, file pattern of the dataset files.
    image_size: int, image size.
    batch_size: int, batch size.
    augmentation: bool, whether to apply augmentation.
    seed: int, seed for shuffling the dataset.
  Returns:
     tf.data.TFRecordDataset.
  """

  filenames = tf.io.gfile.glob(file_pattern)
  dataset = tf.data.TFRecordDataset(filenames)
  dataset = dataset.repeat().shuffle(buffer_size=100, seed=seed)

  # Create a description of the features.
  feature_description = {
      'image/height': tf.io.FixedLenFeature([], tf.int64, default_value=0),
      'image/width': tf.io.FixedLenFeature([], tf.int64, default_value=0),
      'image/channels': tf.io.FixedLenFeature([], tf.int64, default_value=0),
      'image/format': tf.io.FixedLenFeature([], tf.string, default_value=''),
      'image/id': tf.io.FixedLenFeature([], tf.string, default_value=''),
      'image/filename': tf.io.FixedLenFeature([], tf.string, default_value=''),
      'image/encoded': tf.io.FixedLenFeature([], tf.string, default_value=''),
      'image/class/label': tf.io.FixedLenFeature([], tf.int64, default_value=0),
  }

  customized_parse_func = functools.partial(
      _ParseFunction,
      name_to_features=feature_description,
      image_size=image_size,
      augmentation=augmentation,
      unique_landmark_ids=unique_landmark_ids)

  dataset = dataset.map(customized_parse_func)
  dataset = dataset.batch(batch_size)

  return dataset

In [6]:
class GeMPoolingLayer(tf.keras.layers.Layer):
    def __init__(self, p=1., train_p=False):
        super().__init__()
        if train_p:
            self.p = tf.Variable(p, dtype=tf.float32)
        else:
            self.p = p
        self.eps = 1e-6

    def call(self, inputs: tf.Tensor, **kwargs):
        inputs = tf.clip_by_value(inputs, clip_value_min=1e-6, clip_value_max=tf.reduce_max(inputs))
        inputs = tf.pow(inputs, self.p)
        inputs = tf.reduce_mean(inputs, axis=[1, 2], keepdims=False)
        inputs = tf.pow(inputs, 1./self.p)
        return inputs


class DelfArcFaceModel(tf.keras.Model):
    def __init__(self, input_shape, n_classes, margin, logit_scale, feature_size, p=None, train_p=False):
        super().__init__()
        self.backbone = tf.keras.applications.ResNet101(include_top=False, weights="imagenet", input_shape=input_shape)
        #elf.backbone.summary()

        if p is not None:
            self.global_pooling = GeMPoolingLayer(p, train_p=train_p)
        else:
            self.global_pooling = functools.partial(tf.reduce_mean, axis=[1, 2], keepdims=False)
        self.dense1 = tf.keras.layers.Dense(feature_size, activation='softmax', kernel_initializer="glorot_normal")
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.Flatten = tf.keras.layers.Flatten()
        self.arcface = ArcFaceLayer(n_classes, margin, logit_scale)
        
    def call(self, inputs, training=True, mask=None):
        images, labels = inputs
        x = self.extract_feature(images)
        x = self.arcface((x, labels))
        return x
        
    def extract_feature(self, inputs):
        x = self.backbone(inputs)
        x = self.global_pooling(x)
        x = self.Flatten(x)
        x = self.dense1(x)
        x = self.bn1(x)
        return x


class ArcFaceLayer(tf.keras.layers.Layer):
    def __init__(self, num_classes, margin, logit_scale):
        super().__init__()
        self.num_classes = num_classes
        self.margin = margin
        self.logit_scale = logit_scale

    def build(self, input_shape):
        self.w = self.add_weight("weights", shape=[int(input_shape[0][-1]), self.num_classes], initializer=tf.keras.initializers.get("glorot_normal"))
        self.cos_m = tf.identity(tf.cos(self.margin), name='cos_m')
        self.sin_m = tf.identity(tf.sin(self.margin), name='sin_m')
        self.th = tf.identity(tf.cos(math.pi - self.margin), name='th')
        self.mm = tf.multiply(self.sin_m, self.margin, name='mm')

    def call(self, inputs, training=True, mask=None):
        embeddings, labels = inputs
        normed_embeddings = tf.nn.l2_normalize(embeddings, axis=1, name='normed_embd')
        normed_w = tf.nn.l2_normalize(self.w, axis=0, name='normed_weights')

        cos_t = tf.matmul(normed_embeddings, normed_w, name='cos_t')
        sin_t = tf.sqrt(1. - cos_t ** 2, name='sin_t')

        cos_mt = tf.subtract(cos_t * self.cos_m, sin_t * self.sin_m, name='cos_mt')

        cos_mt = tf.where(cos_t > self.th, cos_mt, cos_t - self.mm)

        mask = tf.one_hot(tf.cast(labels, tf.int32), depth=self.num_classes,
                          name='one_hot_mask')

        logits = tf.where(mask == 1., cos_mt, cos_t)
        logits = tf.multiply(logits, self.logit_scale, 'arcface_logist')
        return logits

In [7]:
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model((images, labels), training=True)

        loss = compute_loss(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss * strategy.num_replicas_in_sync)
    train_accuracy(labels, predictions)
    return loss


@tf.function
def distributed_train_steps(training_set_iter, steps_per_call):
    for _ in tf.range(steps_per_call):
        per_replica_losses = strategy.run(train_step, next(training_set_iter))
    # return tpu_strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None)


def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model((images, labels), training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)


@tf.function
def distributed_test_step(images, labels):
    return strategy.run(test_step, args=(images, labels,))

In [39]:
checkpoint_dir = "/home/ubuntu/Dacon/jin/NIA/checkpoint/"
train_tf_records_dir = "/home/ubuntu/Dacon/jin/NIA/tfrecords/train*"
validation_tf_records_dir = "/home/ubuntu/Dacon/jin/NIA/tfrecords/validation*"
test_tf_records_dir = "/home/ubuntu/Dacon/jin/NIA/tfrecords_test/test*"

strategy = tf.distribute.MirroredStrategy()

training_csv_path = os.path.join(data_dir, "train.csv")
train_csv = pd.read_csv(str(training_csv_path))
num_samples = len(train_csv["id"].tolist())
unique_landmark_ids = train_csv["landmark_id"].unique().tolist()
unique_landmark_ids = tf.convert_to_tensor(unique_landmark_ids, dtype=tf.int64)

training_set = CreateDataset(train_tf_records_dir)
training_set = strategy.experimental_distribute_dataset(training_set)

validation_set = CreateDataset(validation_tf_records_dir)
validation_set = strategy.experimental_distribute_dataset(validation_set)

test_set = CreateDataset(test_tf_records_dir)

train_iter = iter(training_set)
validation_iter = iter(validation_set)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [42]:
next(train_iter)[1]

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([243, 243, 243, 117, 243,  86, 117, 157,  91,  16, 233, 233,  91,
       243, 157, 117,  91,  86,  16, 157,  91,  86,  91,  86, 243,  91,
       117,  91,  16, 243,  91, 117])>

In [36]:
next(validation_iter)[1]

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([113,  30, 150, 191, 158,  68,   5, 170,  33, 279,  46, 120, 242,
       240,   3, 149, 110,  36,  21,  51,  34, 214, 302,  54, 287,  55,
       293,  84,  66,  19, 295, 266])>

In [37]:
next(iter(test_set))[1]

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])>

In [9]:
with strategy.scope():
    model = DelfArcFaceModel(
            input_shape=(image_size, image_size, 3), n_classes=len(unique_landmark_ids), margin=margin, logit_scale=logit_scale,
            p=gem_p, train_p=train_p, feature_size=feature_size
        )
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
    train_loss = tf.keras.metrics.Mean(name='train_loss')
    train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
    test_loss = tf.keras.metrics.Mean(name='test_loss')
    test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

    def compute_loss(labels, predictions):
        per_example_loss = loss_object(labels, predictions)
        return tf.nn.compute_average_loss(per_example_loss, global_batch_size=batch_size)

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [ ]:
for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    step = 0
    val_step = 0
    hist = []
    with tqdm(total=int(num_samples)) as pbar:
        while True:
            distributed_train_steps(train_iter, tf.convert_to_tensor(STEPS_PER_TPU_CALL))
            template = 'Epoch {}, Training, Loss: {:.4f}, Accuracy: {:.4f}'
            pbar.set_description(template.format(epoch + 1, train_loss.result(), train_accuracy.result() * 100))
            if step % save_interval == 0:
                if step == 0:
                    model.summary()
                    print()
                    print("\nlearning rate: {}\nmargin: {}\nlogit_scale: {}\ngem_p: {}\ntrain_p{}\n".format(learning_rate, margin, logit_scale, gem_p, train_p))

                checkpoint_path = str(os.path.join(checkpoint_dir, "cp_epoch_{}_step_{}".format(epoch, step)))
                model.save_weights(checkpoint_path)
                print("Model saved to {}".format(checkpoint_path))
            step += batch_size * STEPS_PER_TPU_CALL
            pbar.update(batch_size * STEPS_PER_TPU_CALL)
            if step >= int(num_samples):
                break

    with tqdm(total=int(num_samples)*0.2) as pbar:
        for test_images, test_labels in validation_set:
            distributed_test_step(test_images, test_labels)
            template = 'Epoch {}, Validation, Loss: {:.4f}, Accuracy: {:.4f}'
            pbar.set_description(template.format(epoch + 1, test_loss.result(), test_accuracy.result() * 100))
            val_step += batch_size * STEPS_PER_TPU_CALL
            pbar.update(batch_size)
            if val_step >= int(num_samples)*0.2:
                break

    template = 'Epoch {}, \nTraining Loss: {}, Accuracy: {}\nTest Loss: {}, Accuracy: {}'
    hist.append({"Epoch": epoch,
                 "train_loss": train_loss.result(),
                 "train_accuracy": (train_accuracy.result() * 100),
                 "test_loss": test_loss.result(),
                 "test_accuracy": (test_accuracy.result() * 100)})
    print(template.format(epoch + 1, train_loss.result(), train_accuracy.result() * 100, test_loss.result(), test_accuracy.result() * 100))

In [45]:
data_dirs = '/home/ubuntu/Dacon/cpt_data/landmark/'
test_datagen = ImageDataGenerator(rescale=1./255)

In [46]:
test_data = test_datagen.flow_from_directory(
    directory = data_dirs + 'test/',
    class_mode = 'sparse',
    shuffle = True,
    target_size = (image_size, image_size),
    batch_size = batch_size)

Found 7488 images belonging to 309 classes.


In [47]:
eval_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      name='eval_accuracy')

new_model = DelfArcFaceModel(
            input_shape=(image_size, image_size, 3), n_classes=len(unique_landmark_ids), margin=margin, logit_scale=logit_scale,
            p=gem_p, train_p=train_p, feature_size=feature_size
        )
new_optimizer = tf.keras.optimizers.Adam()

In [52]:
@tf.function
def eval_step(images, labels):
  predictions = model((images, labels), training=False)
  eval_accuracy(labels, predictions)

In [ ]:
checkpoint = tf.train.Checkpoint(optimizer=new_optimizer, model=new_model)
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

for images, labels in test_data:
  eval_step(images, labels)
  print(eval_accuray.result()*100)

print ('전략을 사용하지 않고, 저장된 모델을 복원한 후의 정확도: {}'.format(
    eval_accuracy.result()*100))

In [59]:

next(iter(test_data))[1]

array([161.,  55., 163., 241.,  58.,  29.,  97., 200., 154.,  39., 228.,
        59.,  48., 114., 299.,  69., 305., 271.,  54., 194., 212., 107.,
       204.,  72., 210.,   3., 221.,  64., 170., 248., 303., 128.],
      dtype=float32)

In [61]:
import numpy as np
dummy_labels = np.zeros(32)

In [54]:
for images, labels in tqdm(test_data):
  eval_step(images, labels, dummy_labels)
  print(eval_accuracy.result()*100)

tf.Tensor(0.22727272, shape=(), dtype=float32)
tf.Tensor(0.33987916, shape=(), dtype=float32)
tf.Tensor(0.4518072, shape=(), dtype=float32)
tf.Tensor(0.5536787, shape=(), dtype=float32)
tf.Tensor(0.66429645, shape=(), dtype=float32)
tf.Tensor(0.79291046, shape=(), dtype=float32)
tf.Tensor(0.9021577, shape=(), dtype=float32)
tf.Tensor(1.0107566, shape=(), dtype=float32)
tf.Tensor(1.118713, shape=(), dtype=float32)
tf.Tensor(1.1983776, shape=(), dtype=float32)
tf.Tensor(1.2591912, shape=(), dtype=float32)
tf.Tensor(1.3471408, shape=(), dtype=float32)
tf.Tensor(1.4528509, shape=(), dtype=float32)
tf.Tensor(1.5488338, shape=(), dtype=float32)
tf.Tensor(1.6351745, shape=(), dtype=float32)
tf.Tensor(1.7481884, shape=(), dtype=float32)
tf.Tensor(1.8424855, shape=(), dtype=float32)
tf.Tensor(1.9632565, shape=(), dtype=float32)
tf.Tensor(2.101293, shape=(), dtype=float32)
tf.Tensor(2.2116761, shape=(), dtype=float32)
tf.Tensor(2.375, shape=(), dtype=float32)
tf.Tensor(2.4839745, shape=(), dtype

KeyboardInterrupt: 

In [38]:
tf.train.latest_checkpoint(checkpoint_dir)

'/home/ubuntu/Dacon/jin/NIA/checkpoint/cp_epoch_4_step_16000'

In [25]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.model = model
    
    @tf.function(input_signature=[
      tf.TensorSpec(shape=[None, None, 3], dtype=tf.uint8, name='input_image')
    ])
    
    def call(self, input_image):
        output_tensors = {}
        
        input_image = tf.cast(input_image, tf.float32) / 255.0
        input_image = tf.image.resize(input_image, (384, 384)) 
                
        extracted_features = self.model(tf.convert_to_tensor([input_image], dtype=tf.float32))[0]
        output_tensors['global_descriptor'] = tf.identity(extracted_features, name='global_descriptor')
        return output_tensors

In [ ]:
m = MyModel()
served_function = m.call

tf.saved_model.save(
    m, 
    export_dir="./model", 
    signatures={'serving_default': served_function}
)

In [ ]:
!saved_model_cli show --dir ./my_model/ --all

In [42]:
tf.__version__

'2.2.0'